# Метод Арнолди

## Предварительная информация

### Степенной метод

Пусть дана некоторая матрица $A$ и вектор случайных значений $b$. 
Тогда, посредством многократного возведения этой матрицы и умножения на
$b$, он сходится к собственному вектору $v_1$, соответствующему
максимальному по модулю собственному значению $\lambda_1$, или __спектральному
радиусу__:

$$
    A^k b \xrightarrow[k \to \infty]{} v_1
$$

По-другому это можно считать как:

$$
    b_0 = b \\
    b_1 = Ab_0 \\
    b_2 = Ab_1 = A \cdot Ab_0 = A^2b_0 \\
    ... \\
    b_k = Ab_{k-1}
$$

Собственное значение высчитывается через формулу:

$$
    \lambda_1 = \frac{(b_k,b_{k-1})}{(b_{k-1},b_{k-1})}
$$

### Матрица и подпространство Крылова

Степенной метод позволяет быстро посчитать спектральный радиус на 
разреженной матрице (при хорошей реализации матрично-векторного умножения),
но теряет много информации просто взяв самое последнее значение.

Матрица Крылова сохраняет в себе все предыдущие итерации степенного метода
и имеет вид:

$$
    K_n = \left[ b | Ab | A^2b | ... | A^{n-1}b \right]
$$

Подпространство Крылова образуется из линейной оболочки столбцов матрицы $K_n$:

$$
    \mathcal{K}_n = span \left\{ b, Ab, A^2b, ..., A^{n-1}b \right\}
$$

В пространстве Крылова оперируют разные методы, например метод 
сопряженных градиентов для решения уравнения $Ax = y$ на разреженных матрицах, 
но матрица Крылова, как базис, страдает от проблем, возникающих из свойств 
степенного метода:

* При возведении матрицы в степень значения растут очень быстро и 
  приведут к переполнению в численном подсчете (хотя, как и в степенном методе,
  вектора можно просто нормализовать после каждой итерации);
* Все последующие за начальным вектором $b$ вектора $Ab, A^2b, ...$
  неортогональны и мало подходят в качестве базиса.

## Метод Арнольди

Интуитивно, метод Арнольди является модифицированным процессом Грама-Шмидта, 
который одновременно высчитывает ортонормированный базис Крылова и проекцию матрицы
$A$ на неё.

Метод приводит несамосопряженную матрицу $A$ к верхней Хессенберговой форме $H$.
Формульно это можно записать $A = QHQ^*$ или $AQ = QH$.

Пусть матрица $A$ имеет размер $m \times m$, матрица $Q_n$ имеет размер $m \times n$
для некоторого значения $n < m$ (обозначает размер подпространства Крылова) и 
имеет первые $n$ столбцов матрицы $Q$, а 
$\tilde{H}_n$ - Хессенбергова матрица размером $(n+1) \times n$ - 
левая верхняя часть матрицы $H$. Тогда в матричном виде формула будет иметь вид:

$$
    [A]
    \begin{bmatrix}
    q_1 | & \cdots & | q_n
    \end{bmatrix}
    =
    \begin{bmatrix}
    q_1 | & \cdots & | q_{n+1}
    \end{bmatrix}
    \begin{bmatrix}
    h_{11} & \cdots & h_{1n} \\
    h_{21} & & \vdots \\
     & \ddots  \\
     & & h_{n+1,1}
    \end{bmatrix}
$$

Итоговое выражение имеет вид: $AQ_n = Q_{n+1}\tilde{H}$

Столбец $n$ этого выражения можно высчитать по формуле:

$$
    Aq_n = h_{1n}q_1 + h_{2n}q_2 + ... + h_{n+1,n}q_{n+1}
$$

Здесь столбец $q_{n+1}$ высчитывается рекуррентно из предыдущих значений $q$,
начальный вектор $q_1$ является нормированным вектором $b$.

Следует заметить, что, если выбирать размер подпространства Крылова $n$
больше размера матрицы $m$, то, все последующие вектора $q_{n+1}$ будут
принимать нулевые значения, т.к. все аппроксимации собственных значений
сошлись с их реальными значениями.

В программном виде метод Арнольди будет иметь вид:

In [1]:
import numpy as np

def arnoldi_iteration(A, b, n):
    m = A.shape[0]
    Q = np.zeros((m, n + 1))
    H = np.zeros((n + 1, n))
    q = b / np.linalg.norm(b)
    Q[:, 0] = q

    for k in range(n):
        v = np.dot(A, q)  # Следующий вектор Крылова

        for j in range(k + 1):  # Процесс Грама-Шмидта - вычет проекций
            H[j, k] = np.dot(Q[:, j].conj(), v)
            v -= H[j, k] * Q[:, j]

        H[k+1, k] = np.linalg.norm(v)
        eps = 1e-5 # Из-за плавующей точки итоговый вектор может быть только
        if H[k+1, k] > eps:  # приблизительно нулевым
            q = v / H[k+1, k]
            Q[:, k + 1] = q
        else:  # Если он близок к нулю, то все собственные значения сошлись
            break
    return Q, H

На разреженных матрицах алгоритм работает быстрее прямых методов 
подсчета собственных значений по двум причинам:
* Вовсе не используется дорогая операция умножения матрицы на матрицу,
  а используется простая операция умножения матрицы на вектор, которую
  можно дальше оптимизировать для разреженных матриц;
* Ранее были выделены только части $Q_n$ и $\tilde{H}_n$ от полных матриц,
  $Q$ и $H$. Это возможно, так как метод Арнольди базируется на процессе 
  Грама-Шмидта, который можно прервать. Арнольди будет возвращать $n$
  первых __аппроксимаций__ (необязательно близких к истинным) 
  собственных значений.
  
Точность аппроксимаций растет с размером подпространства Крылова, 
начиная с наибольших по модулю собственных значений (чем больше 
собственное значение, тем быстрее оно аппроксимируется). Аппроксимации
собственных значений будут содержаться в матрице $\tilde{H}_n$,
которые можно получить, например, с помощью QR-алгоритма,
учитывая, что матрица Хессинберга уже приведенная.

Если матрица $A$ самосопряженная, метод сводится к алгоритму Ланцоша, а
Хессенбергова матрица приобретает трехдиагональный вид.

### Пример

В качестве примера простая диагональная матрица, 
где собственные значения лежат на диагонали:

In [2]:
from qr_algo import qr_algorithm
from ipywidgets import interact
import ipywidgets

diagonal = np.diagflat(np.arange(1, 21))

def f(matr, n):
    Q, H = arnoldi_iteration(matr, np.ones((len(matr),)), n)
    # print(f"Hessenberg matrix:\n{np.round(H, 3)}")
    print(f"\nEigenvalues: {qr_algorithm(H[:-1, :], 1e-5)}")
    
interact(f, matr=ipywidgets.fixed(diagonal), n=(1,len(diagonal)))

interactive(children=(IntSlider(value=10, description='n', max=20, min=1), Output()), _dom_classes=('widget-in…

<function __main__.f(matr, n)>

Последний столбец Хессенберговой матрицы нужен только для сохранения реккурентости
и необходимо удалить для подсчета собственных значений.

Другим примером будет матрица, используемая при разработке QR-алгоритма:

In [3]:
matrix2 = np.array([[ 9,  0,  2], 
                    [-6,  4,  4], 
                    [-2, -7,  5]])

interact(f, matr=ipywidgets.fixed(matrix2), n=(1,len(matrix2)))

interactive(children=(IntSlider(value=2, description='n', max=3, min=1), Output()), _dom_classes=('widget-inte…

<function __main__.f(matr, n)>

Меняя размер подпространства Крылова, можно заметить, что собственные значения
сначала стремятся достичь наибольшие по модулю, точность увеличивается с 
размерностью подпространства.

## Применение

Кроме прямого применения на разреженных матрицах для быстрого получения
приведенной матрицы и нахождения на ней собственных значений,
метод Арнольди используется методом _GMRES_ (Generalized Minimal
Residual Method), ищущего численное решение для систем с бесконеченым
множеством решений, а также лежит в основе метода сопряженных градиентов.